<a href="https://colab.research.google.com/github/choeuneheol/python-practice/blob/master/0906Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP_Tutorial_03_Word_Embedding_2

In [ ]:
## Package 설치

 ## datasets(huggingface 자연어처리 dataset)
 ## konlpy (한글 자연어처리 Tool)
 ## tqdm (반복문의 Progress Bar)
 ## glove_python_binary (Glove 임베딩)

In [ ]:
!pip install datasets
!pip install konlpy
!pip install tqdm
!pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.8 MB/s 
     |████████████████████████████████| 212 kB 46.8 MB/s 
     |████████████████████████████████| 120 kB 68.2 MB/s 
     |████████████████████████████████| 115 kB 75.4 MB/s 
     |████████████████████████████████| 127 kB 71.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 8.1 MB/s 
     |████████████████████████████████| 453 kB 71.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 

# 3-0. 데이터셋 준비
 Naver sentiment Movie corpus(nsmc)
 
 영화 리뷰를 별점으로 기준으로 긍정(1), 부정(0)으로 구분한 데이터셋

In [ ]:
from datasets import load_dataset

#nsmc 데이터셋 다운로드
dataset = load_dataset("nsmc")

train_sents = [dataset["train"][idx]["document"] for idx in range(len(dataset["train"]))]
train_labels = [dataset["train"][idx]["label"] for idx in range(len(dataset["train"]))]
test_sents = [dataset["test"][idx]["document"] for idx in range(len(dataset["test"]))]
test_labels = [dataset["test"][idx]["label"] for idx in range(len(dataset["test"]))]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from konlpy.tag import Okt
from tqdm import tqdm

# stopwords 정의
stopwords = ["하다","한","에","와","자","과","걍","잘","좀","는","의","가","이","은","들"]

okt = Okt()

train_nsmc_corpus = []
for sent_i in tqdm(train_sents):
  new_sent_i = sent_i.replace("[^가-힣ㄱ-하-ㅣ ]","")
  tokens = okt.morphs(sent_i, stem=True) #어간 추출
  tokens = [word_i for word_i in tokens if word_i not in stopwords]

  train_nsmc_corpus.append(tokens)

  test_nsmc_corpus = []
  for sent_i in tqdm(test_sents):
    new_sent_i = sent_i.replace("[^가-힣ㄱ-하-ㅣ ]","")
    tokens = okt.morphs(sent_i, stem=True)
    tokens = [word_i for word_i in tokens if word_i not in stopwords]

    test_nsmc_corpus.append(tokens)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  0%|          | 6/150000 [15:41<6535:57:59, 156.87s/it]


KeyboardInterrupt: ignored

# 3-1.Word2Vec (cBow)

word2Vec은 이웃 단어 예측을 통해 단어의 임베딩을 학습하는 기법


CBow 기법 / Skip-Gram  기법



CBow는 주위(Window) 단어를 보고 중심 단어를 예측

In [ ]:
from gensim.models import Word2Vec

CBOW_model = Word2Vec(sentences=train_nsmc_corpus , size = 100 , window = 5 , min_count=5 , workers = 4 , sg =0)
# sg = 0 (CBOW) / sg = 1 (skip gram)

In [ ]:
CBOW_model.wv.most_similar("송강호", topn=5)

In [ ]:
# 사전에 없는 단어
CBOW_model.wv.most_similar("자연어처리", topn=5)

In [ ]:
CBOW_model.wv["송강호"]

In [ ]:
import numpy as np

def sentence_embedding_word2vec(model, sentence, dim=100):
  sent_emb = np.zeros(100)
  n=0

  for word_i in sentence:
    #Vocab에 존재하지 않는 단어의 경우 예외처리
    try:
      word_emb = model.wv[word_i]
      sent_emb += word_emb
      n += 1
    except:
      pass

    